In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import iplot

In [ ]:
# Use .item() to convert from numpy array to dictionary for easier loading
track_data = np.load('tracks.npy', allow_pickle=True).item()
track_df = pd.DataFrame.from_records(track_data)

crthit_data = np.load('crthits.npy', allow_pickle=True).item()
crthit_df = pd.DataFrame.from_records(crthit_data)

match_candidate_data = np.load('match_candidates.npy', allow_pickle=True).item()
match_candidate_df = pd.DataFrame.from_records(match_candidate_data)

crt_plane_df = pd.read_csv('CRT_geometry_walls_xyz_limits.csv')

In [ ]:
match_candidate_df.head()

In [ ]:
# TODO: Split track and CRT hit dataframes into matched and unmatched dataframes. 
# This will make it easier to give (un)matched objects different colors, markers, etc.

matched_tracks = match_candidate_df['track']
matched_crthits = match_candidate_df['crthit']
matched_track_ids  = [matched_tracks[i].id for i,_ in enumerate(matched_tracks)]
matched_crthit_ids = [matched_crthits[i].id for i,_ in enumerate(matched_crthits)]

matched_track_df_mask = track_df['id'].isin(matched_track_ids)
matched_track_df = track_df[matched_track_df_mask]
unmatched_track_df = track_df.drop(matched_track_df.index)
#unmatched_track_df.head()

In [ ]:
def plot_crt_tpc_matches(track_df, crthit_df, crt_plane_df, matched_track_ids, matched_crthit_ids):
    
    traces = []
    
    for index, track in track_df.iterrows():
        
        track_points = track['points']
        track_id = track['id']
        
        track_xs = track_points[:,0]
        track_ys = track_points[:,1]
        track_zs = track_points[:,2]

        track_trace = go.Scatter3d(
            x = track_xs,
            # Flip y and z so that plotly plots the long axis as the z-direction
            y = track_zs,
            z = track_ys,
            mode='markers',
            marker=dict(size=1, color='lightslategrey'),
            name='Track ID ' + str(track_id),
            customdata=np.stack((track_xs, track_ys, track_zs), axis=-1),
            hovertemplate='<b>x</b>: %{customdata[0]:,.2f}<br>' +
                          '<b>y</b>: %{customdata[1]:,.2f}<br>' +
                          '<b>z</b>: %{customdata[2]:,.2f}',
                          showlegend=True)
        
        if track_id in matched_track_ids:
            track_trace.marker.color = 'red'
        
        traces.append(track_trace)
        
    crthit_id = crthit_df['id']
    crthit_xs = crthit_df['position_x']
    crthit_ys = crthit_df['position_z']
    crthit_zs = crthit_df['position_y']
    
    matched_crthit_mask = crthit_id.isin(matched_crthit_ids)
    marker_color = np.array(['lightslategrey'] * len(crthit_xs)).astype(object)
    marker_color[matched_crthit_mask] = 'red'
    
    crthit_trace = go.Scatter3d(
        x = crthit_xs,
        y = crthit_ys,
        z = crthit_zs,
        mode='markers',
        marker=dict(size=2, color=marker_color),
        customdata=np.stack((crthit_id, crthit_xs, crthit_ys, crthit_zs), axis=-1),
        #name=[f"CRT Hit ID {id}" for id in crthit_id],
        hovertemplate='<b>ID</b>: %{customdata[0]:i}<br>' +
                      '<b>x</b>: %{customdata[1]:,.2f}<br>' +
                      '<b>y</b>: %{customdata[2]:,.2f}<br>' +
                      '<b>z</b>: %{customdata[3]:,.2f}',
                      showlegend=True)
    
    traces.append(crthit_trace)
    
    fig = go.Figure(traces)
    
    for index, row in crt_plane_df.iterrows():
        x_min, x_max, y_min, y_max, z_min, z_max = row[['Xmin (cm)', 'Xmax (cm)', 'Ymin (cm)', 'Ymax (cm)', 'Zmin (cm)', 'Zmax (cm)']]
        plane_region = row['Region Name']
        thin_axis = row['thin range (x=0, y=1, z=2)']
        x_values = [[x_min, x_max], [x_min, x_max]]
        y_values = [[z_min, z_min], [z_max, z_max]]
        z_values = [[y_min, y_min], [y_max, y_max]]
        # Plotly surfaces are weird...have to change the xyz values
        # depending on which axis the plane is aligned with
        if thin_axis == 0:
            x_values = [[x_min, x_min], [x_max, x_max]]
            y_values = [[z_min, z_max], [z_min, z_max]]
            z_values = [[y_min, y_min], [y_max, y_max]]
        plane_trace = go.Surface(
            x=x_values, y=y_values, z=z_values,
            opacity=0.2,
            # A pale green-ish color to distinguish from anode/cathode planes
            colorscale = [[0, 'rgb(224, 255, 255)'], [1, 'rgb(224, 255, 255)']],
            hoverinfo='none',
            name=plane_region
        )
        fig.add_trace(plane_trace)
    
    surf_x0 = -210.215*np.ones(100)
    surf_x1 = 210.215*np.ones(100)
    surf_y = np.linspace(-181.6, 134.96, 100)
    surf_z = np.linspace(-894.9505, 894.9505, 100)
    surf_ytest = surf_z
    surf_ztest = surf_y
    #surf_z = np.linspace(232, 432, 100)                                                                                                                   
    rc = np.random.rand(100, 100)
    mycolorscale = [[0, '#aa9ce2'],[1, '#aa9ce2']]
                                                                                                                               
    plane0 = go.Surface(
        x=surf_x0, y=surf_ytest, z=np.array([surf_ztest] * len(surf_x0)), 
        colorscale=mycolorscale, opacity=0.25, showscale=False, hoverinfo='none', name='Plane 0')
    plane1 = go.Surface(
        x=surf_x1, y=surf_ytest, z=np.array([surf_ztest] * len(surf_x1)), 
        colorscale=mycolorscale, opacity=0.25, showscale=False, hoverinfo='none', name='Plane 1')
    
    fig.add_trace(plane0)
    fig.add_trace(plane1)
    fig.update_layout(scene = dict(
        xaxis_title='x [cm]',
        yaxis_title='z [cm]',
        zaxis_title='y [cm]'),
        height=1000, width=1300
    )
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    fig.update_scenes(aspectmode='data')
    iplot(fig)



In [ ]:
plot_crt_tpc_matches(track_df, crthit_df, crt_plane_df, matched_track_ids, matched_crthit_ids)